# Deterministic Global Optimization for Concentrating Solar Thermal Hybridization

This example comes from M.D. Stuber. A Differentiable Model for Optimizing Hybridization of Industrial Process Heat Systems with Concentrating Solar Thermal Power. _Processes_, 6(7), 76 (2018) DOI: [10.3390/pr6070076](https://doi.org/10.3390/pr6070076)

In this example, we seek to determine the optimal thermal energy storage capacity and parabolic trough solar array aperture area that maximizes the lifecycle savings associated with augmenting a conventional natural gas industrial process heat system.  Here, we use user-defined functions, the JuMP modeling language, the EAGO spatial branch-and-bound algorithm with custom upper- and lower-bounding procedures, and the IPOPT algorithm for solving the bounding subproblems.

We will solve the optimal design problem for Firebaugh, CA with the commercial fuel rate and constant industrial process heat demand ($\xi=0$).

Note: This example corresponds to a $\bar{q}_p=10^4$ kW thermal demand.  In the paper, it is stated that a $\bar{q}_p=10^5$ kW thermal demand was studied.  However, this is an error as all studies in the paper were conducted for a $\bar{q}_p=10^4$ kW thermal demand.

In [1]:
using JuMP, EAGO, Ipopt

We will also use the CSV package because our solar resource data (downloaded from the NSRDB) is in a CSV file.

In [2]:
using CSV, DataFrames

The code is organized as follows:  
1. We import the solar resource data for the region we are concerned with.  By default, this is the typical meteorological year (TMY) hourly data (8760 data points) which includes the geographic information (coordinates and timezone) as well as the direct normal irradiance (DNI).  Combined, we can model the process of concentrating the solar radiation and collecting it as heat in our system.  
2. Once we have the resource data, we call the `PTCmodel` function from "PTCmodel.jl" to simulate the specific thermal power of the solar concentrating technology (a parabolic trough in this case) in units of kW/m$^2$ given our geographic region and incident angle data.  This function calls `solarAngles` from "solarAngles.jl" to calculate the angles of the direct solar radiation incident to the concentrator aperture with respect to each hour in the TMY data.
4. We define a function closure for the solar fraction calculated by `iphProcessSmooth` from "iphProcessSmooth.jl", which simulates the performance of the full solar concentrator and thermal storage system hybridized with the industrial process heat system.  The solar fraction is calculated by Eq. 12 in the paper.
5. We define the custom upper- and lower-bounding optimization subproblems for the spatial branch-and-bound algorithm.  The lower-bounding problem uses the convex capital model for its objective function (Eq. 16 in the paper), and the upper-bounding problem uses the nonconvex capital model as its objective function (Eq. 14 in the paper).
6. We set up the JuMP model with the EAGO optimizer (with custom bounding procedures) and we solve it.

In [3]:
include("smoothMinMaxAbs.jl")
include("solarAngles.jl")
include("PTCmodel.jl")
include("iphProcessSmooth.jl")
include("lifecycleCost.jl")
include("smoothMinMaxAbs.jl")

# Step 1: Read the data into a table and extract the appropriate data into a vector
input_file = "FirebaughTMY_Julia.csv"
solData = CSV.File(input_file) |> DataFrame
yData = convert(Array{Float64,1}, solData[:,7])

# Step 2: Get the specific thermal power potential for the region [kW/m^2]
q = PTCmodel(yData, -8, 36.85, -120.46)

# Step 3: Define the solar fraction closure
SolarFrac(xts,xa) = iphProcessSmooth(q, xts, xa);

Step 4 is a bit more complicated as we need to define the custom bounding procedures for the branch-and-bound algorithm.  First, we define an extension type called $\texttt{SolarExt}$ which allows EAGO to dispatch to the custom bounding routines we plan on defining.  

In [4]:
import EAGO: Optimizer, GlobalOptimizer
struct SolarExt <: EAGO.ExtensionType end

Now, we define the custom lower-bounding problem with the convex capital model.

In [5]:
# Lower Problem Definition
import EAGO.lower_problem!
function lower_problem!(t::SolarExt, opt::GlobalOptimizer)
    
    # Get active node
    n = opt._current_node
    
    # Create model and add variables
    mL = JuMP.Model(JuMP.optimizer_with_attributes(Ipopt.Optimizer,
                    "tol" => 1.0e-3,
                    "print_level" => 0))
    xL = n.lower_variable_bounds
    xU = n.upper_variable_bounds
    @variable(mL, xL[i] <= x[i=1:2] <= xU[i])
    
    # Define convex lifecycle savings for use in the objective function
    flcsC(xts,xa) = lifecycleCost(q, xL, xU, xts, xa, :convex)
    
    # Register the convex lifecycle savings and solar fraction closure with our model
    JuMP.register(mL, :flcsC, 2, flcsC, autodiff=true)
    JuMP.register(mL, :SolarFrac, 2, SolarFrac, autodiff=true)

    # Define the nonlinear objective and constraint
    @NLobjective(mL, Max, flcsC(x[1], x[2]))
    @NLconstraint(mL, g1, SolarFrac(x[1], x[2]) >= 0.0) # Declare constraints
    
    # Optimize the model
    JuMP.optimize!(mL)

    # Interpret status codes for branch-and-bound
    tstatus = MOI.get(mL, MOI.TerminationStatus())
    pstatus = MOI.get(mL, MOI.PrimalStatus())
    
    if EAGO.local_problem_status(tstatus, pstatus) == EAGO.LRS_FEASIBLE
        opt._lower_objective_value = -1.0*(JuMP.objective_value(mL)-1e-4) # Multiplied by -1 because EAGO expects "Min"
        opt._lower_solution = JuMP.value.(x)
        opt._lower_feasibility = true
        opt._cut_add_flag = false
    else
        opt._lower_feasibility = false
        opt._lower_objective_value = Inf
        opt._cut_add_flag = false
    end
    
    return
end;

Now, we define the upper-bounding problem with the nonconvex capital pricing model.

In [6]:
# Upper Problem Definition
import EAGO.upper_problem!
function upper_problem!(t::SolarExt,opt::GlobalOptimizer)
    
    # Get active node
    n = opt._current_node
    
    # Create model and add variables
    mU = JuMP.Model(JuMP.optimizer_with_attributes(Ipopt.Optimizer,
                    "tol" => 1.0e-3,
                    "print_level" => 0))
    xL = n.lower_variable_bounds
    xU = n.upper_variable_bounds
    @variable(mU, xL[i] <= x[i=1:2] <= xU[i])
    
    # Define nonconvex lifecycle savings for use in the objective function
    flcs(xts,xa) = lifecycleCost(q, xL, xU, xts, xa, :nonconvex)
    
    # Register the convex lifecycle savings and solar fraction closure with our model
    JuMP.register(mU, :flcs, 2, flcs, autodiff=true)
    JuMP.register(mU, :SolarFrac, 2, SolarFrac, autodiff=true)

    # Define nonlinear function
    @NLobjective(mU, Max, flcs(x[1], x[2]))
    @NLconstraint(mU, g1, SolarFrac(x[1], x[2]) >= 0.0)# declare constraints
    JuMP.optimize!(mU)

    # Interpret status codes for branch-and-bound
    tstatus = MOI.get(mU, MOI.TerminationStatus())
    pstatus = MOI.get(mU, MOI.PrimalStatus())

    if EAGO.local_problem_status(tstatus, pstatus) == EAGO.LRS_FEASIBLE
        opt._upper_objective_value = -1.0*JuMP.objective_value(mU) # Multiplied by -1 because EAGO expects "Min"
        opt._upper_solution = JuMP.value.(x)
        opt._upper_feasibility = true
    else
        opt._upper_feasibility = false
        opt._upper_objective_value = Inf
        opt._cut_add_flag = false
    end
    return
end;

Since we have defined custom bounding routines, we'll disable some unnecessary EAGO subroutines.

In [7]:
import EAGO: preprocess!, postprocess!, cut_condition
function EAGO.preprocess!(t::SolarExt, x::GlobalOptimizer)
    x._preprocess_feasibility = true
    return
end
function EAGO.postprocess!(t::SolarExt, x::GlobalOptimizer)
    x._postprocess_feasibility = true
    return
end
EAGO.cut_condition(t::SolarExt, x::GlobalOptimizer) = false

Now, we define the JuMP model and the variables (and bounds).  We must specify that our new custom extension $\texttt{SolarExt}$ of EAGO's default routines should be used and that we will be branching on both of our decision variables.  The latter is required for custom routines since no expressions will be provided to the EAGO optimizer and therefore it cannot infer which variables should be branched on. 

In [8]:
factory = () -> EAGO.Optimizer(SubSolvers(; t=SolarExt()))
m = JuMP.Model(optimizer_with_attributes(factory,
                                "relative_tolerance" => 1e-2,
                                "verbosity" => 1,
                                "output_iterations" => 1, 
                                "branch_variable" => Bool[true; true],
                                ))

x_L = [0.001, 0.01]
x_U = [16.0, 60000.0]
@variable(m, x_L[i] <= x[i=1:2] <= x_U[i])
JuMP.register(m, :SolarFrac, 2, SolarFrac, autodiff=true)
@NLconstraint(m, g1, SolarFrac(x[1], x[2]) >= 0.0);

Now, let's solve the problem and print the results.

In [9]:
optimize!(m)

println("xts* = ", JuMP.value(x[1]), " xa* = ",
         JuMP.value(x[2])," f* = ",-1.0*JuMP.objective_value(m)," SF* = ",
         SolarFrac(JuMP.value(x[1]),JuMP.value(x[2])))
TermStatus = JuMP.termination_status(m)
PrimStatus = JuMP.primal_status(m)
println("Algorithm terminated with a status of $TermStatus and a result code of $PrimStatus")


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

-----------------------------------------------------------------------------------------------------------------------------
|  Iteration #  |     Nodes    | Lower Bound  |  Upper Bound  |      Gap     |     Ratio    |     Time     |    Time Left   |
-----------------------------------------------------------------------------------------------------------------------------
|            1  |            2 |   -7.531E+06 |    3.084E+02 |   7.531E+06 |    1.000E+00 |    9.286E+00 |    3.591E+03 |
|            2  |            3 |   -7.531E+06 |    3.084E+02 |   7.531E+06 |    1.000E+00 |    2.178E+01 |    3.5

So, we find the guaranteed global optimal solution is $x_{ts}^*=11.72$ h and $x_{a}^*=43,615.2$ m$^2$ with a solar fraction of $SF_s^*=0.698$ and an optimal solution value of $f^*_{disc}=7.320$ million dollars.  This is exactly what is found in Table 1 in the paper for Firebaugh, CA and the concave capital cost model.